In [4]:
max1demand=6   #use 1 more than actual demand
max2demand=6    #use 1 more than actual demand
max3demand=6   #use 1 more than actual demand
max4demand=6
sparti=[]  #start with null
spartj=[]  #start with null
spartk=[]  #start with null
spartl=[]  #start with null
spartm=[]  #start with null
spartn=[]  #start with null
sidx=[]    #start with null
s=1        #start number for loop
for i in 1:8          #Create an index to call
    for j in 1:max1demand
        for k in 1:max2demand
            for l in 1:max3demand
                for m in 1:max4demand
                    for n in 1:4 #4 regions
          #statei(i+1) = s
            sparti = push!(sparti,i)  #for i in ijkl add to sparti definition
            spartj = push!(spartj,j) #for j in ijkl add to spartj definition
            spartk = push!(spartk,k) #for k in ijkl add to spartk definition
            spartl = push!(spartl,l) #for l in ijkl add to spartl definition
            spartm = push!(spartm,m) #for l in ijkl add to spartl definition
            spartn = push!(spartn,n) #for l in ijkl add to spartl definition
            sidx= push!(sidx,s)      #create index in sidx
            s=s+1
                        
                    end
                end
            end
        end
    end
end

idxidle= (sparti.<5) .& (spartj+spartk+spartl+spartm .>4)   #index all ijkl where idling cannot occur
idxregion1= (sparti.==5) .& (spartj.==1)      #index all ijkl where region 1 is active but there are no calls in region 1 to service
idxregion2= (sparti.==6) .& (spartk.==1)      #index all ijkl where region 2 is active but there are no calls in region 2 to service
idxregion3= (sparti.==7) .& (spartl.==1)      #index all ijkl where region 2 is active but there are no calls in region 3 to service
idxregion4= (sparti.==8) .& (spartm.==1)      #index all ijkl where region 2 is active but there are no calls in region 4 to service
#idxZoneFrom= (sparti.==1) .& (spartn.==(2|3|4)) .| (sparti.==2) .& (spartn.==(1|3|4)) .| (sparti.==3) .& (spartn.==(1|2|4)) .| (sparti.==4) .& (spartn.==(1|2|3))  #idx where zone from cannot occur

idxDEL=idxidle .| idxregion1 .| idxregion2 .| idxregion3 .| idxregion4 #.|idxZoneFrom #index of the previous 6 indices

deleteat!(sparti, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartj, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartk, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartl, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartm, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartn, BitArray(idxDEL)) #delete idxDEL        
sidx= [1:length(sparti);]       #number of states

17296-element Vector{Int64}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
 17285
 17286
 17287
 17288
 17289
 17290
 17291
 17292
 17293
 17294
 17295
 17296

In [2]:
λ=0 #Define overall Arrival Rate for every zone
N=["λ" "μ1" "μ2" "μ3" "Throughput" "Deliveries/Hour" "Avg. # in Sys" "Avg. Time in Sys" "% of Time 0 calls" "% of Time 1 call" "Compliance Rate"]

for i in 1:3
     λ= λ+.125
     μ1=5.5
for j in 1:7
      μ1=μ1-.5
#Add Arrival Code
λidx1=zeros(length(sidx))          
λidx2=zeros(length(sidx)) 
λidx3=zeros(length(sidx)) 
λidx4=zeros(length(sidx)) 
μ2=.9*μ1
μ3=.85*μ1
μMatr= [μ1  μ2  μ2 μ3;     #Zone 1 to other zone rates
        μ2  μ1  μ3 μ2;    #Zone 2 to other zone rates
        μ2  μ3  μ1 μ2;     #Zone 3 to other zone rates
        μ3  μ2  μ2 μ1;]    #Zone 4 to other zone rates
        
matrix= zeros(length(sidx),length(sidx))
μMat=zeros(length(sidx))


for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
    
    
 #   Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2
        nextCall3= Call3
        nextCall4= Call4
        
        if Zoneflag <=4
            nextZoneflag = 5 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom            
        end      
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].= matrix[nexts,s] .+ λ
        matrix[s,s]= matrix[s,s] - λ
        λidx1[s]=1
    end

    
 #   Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1
        nextCall3= Call3
        nextCall4= Call4

        if  Zoneflag <=4
            nextZoneflag = 6 #don't use double equal
            nextRegionFrom=Zoneflag
        else
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  

         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s] .+ λ
         matrix[s,s]= matrix[s,s] - λ
        λidx2[s]=1
    end

 #   Arrival at region 3
    if Call3<max3demand
        nextCall1= Call1
        nextCall2= Call2
        nextCall3= Call3+1
        nextCall4= Call4

        
        if Zoneflag <=4 
            nextZoneflag = 7 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  
         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s].+λ
         matrix[s,s]= matrix[s,s].-λ
        λidx3[s]=1
    end  
    
 #   Arrival at region 4
    if Call4<max4demand
        nextCall1= Call1
        nextCall2= Call2
        nextCall3= Call3
        nextCall4= Call4+1

        
        if Zoneflag <=4 
            nextZoneflag = 8 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag= Zoneflag
            nextRegionFrom=RegionFrom
        end  
         idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
         nexts= sidx[idx] 
         matrix[nexts,s].=matrix[nexts,s].+λ
         matrix[s,s]= matrix[s,s].-λ
        λidx4[s]=1
    end  

end   
#     Service from Region to Region
    
for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    Call3 = spartl[s]
    Call4 = spartm[s]
    RegionFrom= spartn[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nextCall3=0
    nextCall4=0
    nexts=0
    nextRegionFrom=0
    RegionService= 0
 

    if Zoneflag == 5                #Formulation for Finish Zone Policy
        if Call1-1 >=max(Call2,Call3,Call4)/2 && Call1-1 >1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
        elseif Call1-1 >max(Call2,Call3,Call4)/2 && Call1-1 ==1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
#             println(nextZoneflag)
        elseif max(Call2, Call3, Call4)==Call2
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
            #μidx=push!(μidx,1)
            
        elseif max(Call2, Call3, Call4)==Call3
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1
            #μidx=push!(μidx,1)
                        
        else
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1-1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=1               
        end
 
    
    elseif Zoneflag == 6                #Formulation for Finish Zone Policy

        if Call2-1 >=max(Call1,Call3,Call4)/2 && Call2-1 > 1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=2                        
        elseif max(Call1, Call3, Call4)==1 && Call2-1 ==1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=2
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=2                 
        elseif max(Call1, Call3, Call4)==Call1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=2                             
        elseif max(Call1, Call3, Call4)==Call4
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=2                   
        else
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2-1
            nextCall3=Call3
            nextCall4=Call4
            nextRegionFrom=2                 
            #μidx=push!(μidx,1)        
        end

  
    elseif Zoneflag == 7                #Formulation for Finish Zone Policy

        if Call3-1 >=max(Call1,Call2,Call4)/2 && Call3-1 > 1
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=3                        
        elseif max(Call1, Call2, Call4)==1 && Call3-1 ==1
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=3
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=3  
        elseif max(Call2, Call1, Call4)==Call4
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=3              
        elseif max(Call2, Call1, Call4)==Call1
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=5
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=3              
        else
            RegionService=μMatr[3,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3-1
            nextCall4=Call4
            nextRegionFrom=3              
            #μidx=push!(μidx,1)        
        end     
                
    elseif Zoneflag == 8                #Formulation for Finish Zone Policy

        if Call4-1 >=max(Call1,Call2,Call3)/2 && Call4-1 > 1
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=8
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=4              
            #μidx=push!(μidx,1)
        elseif max(Call1, Call2, Call3)==1 && Call4-1 ==1
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=4
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=4             
#             println(nextZoneflag)
        elseif max(Call2, Call3, Call1)==Call3
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=7
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=4             
            #μidx=push!(μidx,1)
                        
        elseif max(Call2, Call3, Call1)==Call2
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag=6
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=4             
            #μidx=push!(μidx,1)        
        else
            RegionService=μMatr[4,RegionFrom]
            nextZoneflag= 5
            nextCall1=Call1
            nextCall2=Call2
            nextCall3=Call3
            nextCall4=Call4-1
            nextRegionFrom=4             
            #μidx=push!(μidx,2)
        end        

                    
    else
        RegionService= 0
        nextZoneflag=Zoneflag
        nextCall1=Call1
        nextCall2=Call2
        nextCall3=Call3
        nextCall4=Call4
        nextRegionFrom=RegionFrom
        #μidx=push!(μidx,0)
    end 

    idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextCall3) .& (spartm.==nextCall4) .& (spartn.==nextRegionFrom)
    nexts= sidx[idx]     
    matrix[nexts,s]=matrix[nexts,s].+RegionService
    matrix[s,s]= matrix[s,s].- RegionService  
    μMat[s,1]= μMat[s,1] .+ RegionService
end
matrix[length(sidx),:].= 1; #prep Ahat for matrix inversion
inverse=inv(matrix) #invert Ahat matrix
sum(inverse[:,length(sidx)]) #validate that sum of π equals 1
X=inverse[:,length(sidx)] #extract π from matrix
X=round.(X,digits=10); #round to 10 digits
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
E=spartm[1:length(sidx)]
F=spartn[1:length(sidx)]
M=hcat(sidx,A,B,C,D,E,F,X) 
Throughput=(λ)*sum(M[λidx1.==1,8])+(λ)*sum(M[λidx2.==1,8])+(λ)*sum(M[λidx3.==1,8])+(λ)*sum(M[λidx4.==1,8])
Gain=μMat'*X
L=0
for s in 1:length(sidx)
  L=L+(B[s]+C[s]+D[s]+E[s]-4)*X[s]
end
W=L/Throughput 
Midx0= (spartj .+ spartk .+ spartl.+spartm).==4
ZeroCall=sum(M[Midx0.==1,8])
#Percentage of time with one calls
Midx1= (spartj .+ spartk .+ spartl.+spartm).==5
OneCall=sum(M[Midx1.==1,8])
CR=ZeroCall+OneCall
N=[N; λ μ1 μ2 μ3 Throughput Gain L W ZeroCall OneCall CR]
end
end


In [3]:
using DataFrames
df= DataFrame(N[2:Int(length(N)/11),:], :auto)



,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0.125,5.0,4.5,4.25,0.499563,0.499563,0.123162,0.24654,0.890162,0.0978563
2,0.125,4.5,4.05,3.825,0.499452,0.499452,0.138628,0.277559,0.877994,0.107239
3,0.125,4.0,3.6,3.4,0.499292,0.499292,0.15851,0.317468,0.8628,0.118552
4,0.125,3.5,3.15,2.975,0.499052,0.499052,0.185,0.370702,0.843299,0.132419
5,0.125,3.0,2.7,2.55,0.498665,0.498665,0.222007,0.445203,0.817366,0.149728
6,0.125,2.5,2.25,2.125,0.497983,0.497983,0.277226,0.556698,0.781221,0.171713
7,0.125,2.0,1.8,1.7,0.496614,0.496614,0.368017,0.741053,0.727469,0.199847
8,0.25,5.0,4.5,4.25,0.995965,0.995965,0.277226,0.278349,0.781221,0.171713
9,0.25,4.5,4.05,3.825,0.994857,0.994857,0.31636,0.317995,0.757253,0.184928


In [8]:
using CSV
CSV.write("C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\4ZoneFIZFZ_AllArrival_AmbStats10FEB_1.csv",df)

"C:\\Users\\alexb\\Documents\\NCSU\\501\\Thesis\\4ZoneFIZFZ_AllArrival_AmbStats10FEB_1.csv"